In [1]:
import nbimporter

import sys
sys.path.append('../')

from module import climate
from module import effect_coefficient
from module import sun_position
from module import direct_solar_area
from module import shading_effect_factor

Importing Jupyter notebook from ..\module\climate.ipynb
Importing Jupyter notebook from ..\module\sun_position.ipynb
Importing Jupyter notebook from ..\module\effect_coefficient.ipynb
Importing Jupyter notebook from ..\module\direct_solar_area.ipynb
Importing Jupyter notebook from ..\module\shading_effect_factor.ipynb


# 日よけ効果係数算定ツール  Main

- 一次目標：日よけ効果係数算定ツールのpython上での再現


## X. 日よけ効果係数算定ツール本体＋入出力


### X.1 本体プログラム

In [2]:
""" 日よけ効果係数計算プログラム本体 """

def Calc_ShadingCorrectionFactor(Path00, FileName00, FileName01, ClimateZone, NDT, etaID, Azimuth, WSSize ):

    # FileName00 は使用していない。

    # 引数(の例)
    #    Path00 = "./SCFConfig01/"  # 設定ファイルのあるパス
    #    FileName01 = "IncidentAngleCharacteristics.csv"   # 窓ガラスの入射角特性ファイル
    #    ClimateZone = 6            # 地域区分(1～8地域, 他、ユニークなID設定可)
    #    NDT = 6                    # 1時間の分割数,ツールの標準は6分割
    #    Azimuth = "南"             # 窓面の方位を16方位か、角度(-180°< Azimuth <= 180°)で入力
    #    etaID = 1                  # 入射角特性のデータセットのID
    #    WSSize = [1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2]
    # WSSize = [X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym]
    # WSSize:窓および日よけの寸法一式
    
    # \Zone.csv から地点データの読み込み
    c = climate.Climate(ClimateZone, NDT)
    
    # 窓の方位(-180~180)
    Azwj = sun_position.calc_Azwj(Azimuth)
    
    # 天空日射の効果係数 gammayp (天空の形態係数の2倍)
    gammayp = effect_coefficient.calc_gammayp(WSSize)

    # 反射日射の効果係数 gammaym (地面の形態係数の2倍)
    gammaym = effect_coefficient.calc_gammaym(WSSize)
    
    # 窓ガラスの入射角特性
    iac = climate.get_incident_angle_characteristics()
    
    # 窓面積 Awj, m2
    Awj = WSSize[1] * WSSize[8]    # Awj = X2 * Y2
    
    """ リストの初期設定 """
    SCF01 = [[[0 for j in range(NDT)] for i in range(8761)] for h in range(2)]
    
    for Hour00 in range(8760): # ← 12/31 24時は回さないのでHour00=8759がループの最後
        
        for MM in range(c.ndt): # MM:1時間の内の(1/NDT)間隔の順番, 正時がMM=0, MM=0～NDT-1
            
            # 太陽高度とその正弦,余弦の計算
            sinh = c.calc_sinh(Hour00, MM) #
            cosh = c.calc_cosh(Hour00, MM) #
            hsdt = c.calc_hsdt(Hour00, MM) #
            
            # 太陽方位角 Azsdt (deg) の計算
            Azsdt = c.calc_azsdt(Hour00, MM) #
            
            # 法線面直達日射量 Sddhm (kcal/m2), 水平面天空日射量 Ssdhm (kcal/m2)
            Sddhm = c.calc_sddhm(Hour00, MM) #
            Ssdhm = c.calc_ssdhm(Hour00, MM) #
            
            # 窓面の法線ベクトルと太陽位置とのなす水平面上の角度 Azwjdt (deg)
            Azwjdt = sun_position.calc_Azwjdt(Azwj, Azsdt)
            
            # 直達入射の窓面への入射角の余弦 costheta
            costheta = climate.calc_costheta(Azwjdt, cosh)
   
            # 入射角特性 etajdt
            etajdt = climate.calc_etajdt(costheta, iac.eta_kk)

            # :直達日射が窓に射す部分の面積 Ax, m2
            Ax = direct_solar_area.calc_Ax(WSSize, Azwjdt, hsdt)

            """ 日よけ効果係数算定式の時刻TTにおける分子分母(E.2) """
                # SCF00[0]:(2)式の分子への加算分(窓面積をかけた値として)     
                # SCF00[1]:(2)式の分母への加算分(窓面積をかけた値として)
            [SCF01[0][Hour00][MM],SCF01[1][Hour00][MM]] \
                = shading_effect_factor.calc_SCF00(Sddhm, Ssdhm, etajdt, iac.eta_isr,
                                                   costheta, sinh
                                       , Awj, Ax, gammayp, gammaym)
         
    SCF = shading_effect_factor.Output_ShadingCorrectionFactor(c, NDT, SCF01)
    
    return SCF